In [28]:
import tabula
import pandas as pd
MEN_PATH = "/app/fixed_results_m.pdf"
WOMEN_PATH = "/app/results_w.pdf"
MIXED_PATH = "/app/results_mixed.pdf"


def load_tables(path):
    return tabula.read_pdf(path, pages='all', stream=True)

In [29]:
men_tables, women_tables, mixed_tables = load_tables(MEN_PATH), load_tables(WOMEN_PATH), load_tables(MIXED_PATH)

In [30]:
def clean_split_name_from_time(df):
    # Cleans dfs that have columns like 'Firstname-Lastname22:31:02.3'
    # Define regex pattern for matching the time in hh:mm:ss.x format
    time_pattern = r"(\d{2}:\d{2}:\d{2}\.\d)"
    replace_pattern = r"\d{2}:\d{2}:\d{2}\.\d"
    # Extract the time from the original column using regex and create a new column
    df.insert(5, 5, df.iloc[:, 2].str.extract(time_pattern))
    # Remove the time from the original column, leaving just the name
    df.iloc[:, 2] = df.iloc[:, 2].str.replace(
        replace_pattern, "", regex=True).str.strip()
    # Reorder columns 3-5 to move column index 5 to index 3
    df = df.iloc[:, [0, 1, 2, 5, 3, 4]]
    return df

def clean_move_name_from_time(df):
    # Cleans dfs that have columns like 'Firstname-Lastname22:31:02.3'
    # Define regex pattern for matching the time in hh:mm:ss.x format
    time_pattern = r"(\d{2}:\d{2}:\d{2}\.\d)"
    replace_pattern = r"\d{2}:\d{2}:\d{2}\.\d"
    # Extract the time from the original column using regex and create a new column
    df.iloc[:,3] = df.iloc[:, 2].str.extract(time_pattern)
    # Remove the time from the original column, leaving just the name
    df.iloc[:, 2] = df.iloc[:, 2].str.replace(
        replace_pattern, "", regex=True).str.strip()
    return df

def load_run_df(tables):
    # Loads extracted tables into df and puts things in the right order
    columns = tables[0].columns
    # Create an empty DataFrame with column names from the first DataFrame
    run_df = pd.DataFrame(columns=columns)
    for df in tables:
        df = df.T.reset_index().T.reset_index(drop=True)
        # Name and time shoved together, missing time column
        if len(df.columns) != len(columns):
            df = clean_split_name_from_time(df)
        # Name and time shoved together, time column NaN or garbage data
        elif ':' in df.iloc[0,2]:
            df = clean_move_name_from_time(df)
        df.columns = columns
        df = df.loc[df.ne(df.columns).any(axis=1)]
        run_df = pd.concat([run_df, df], ignore_index=True)

    run_df = run_df.drop(columns=['GPos'])
    print(
        f'Processed {len(run_df)} total entries, {len(run_df[run_df.isna().any(axis=1)])} have NaNs')
    run_df = run_df.dropna(subset='Zeit')
    print(
        f'After dropping time NaNs {len(run_df[run_df.isna().any(axis=1)])} NaNs left')
    return run_df

In [31]:
men_run_df, women_run_df, mixed_run_df = load_run_df(men_tables), load_run_df(women_tables), load_run_df(mixed_tables)

Processed 3695 total entries, 16 have NaNs
After dropping time NaNs 2 NaNs left
Processed 1977 total entries, 5 have NaNs
After dropping time NaNs 0 NaNs left
Processed 5719 total entries, 21 have NaNs
After dropping time NaNs 2 NaNs left


In [32]:
men_run_df[men_run_df.isna().any(axis=1)]

,Pos,Nr,Name,Zeit,Firma
659,660.0,50060,Carsten Smidt,00:30:42.6,NaN
2346,2347.0,50049,Bjoern Bischoff,00:38:23.4,NaN


In [33]:
mixed_run_df.head()

,Pos,Nr,Name,Zeit,Firma
0,1.0,4911,Michael Majewski,00:19:16.7,hkk Krankenkassse
1,2.0,4022,Leonard Maisch,00:20:12.7,DFKI GmbH - RIC
2,3.0,7001,No Name,00:20:12.8,Vitakraft pet care GmbH & Co. KG
3,4.0,6074,Sven Zabel,00:21:34.9,OHB SE
4,5.0,2022,Patrick Dekorsi,00:21:44.2,ACTEGA DS GmbH


In [34]:
men_run_df.head()

,Pos,Nr,Name,Zeit,Firma
0,1.0,4911,Michael Majewski,00:19:16.7,hkk Krankenkassse
1,2.0,4022,Leonard Maisch,00:20:12.7,DFKI GmbH - RIC
2,3.0,6074,Sven Zabel,00:21:34.9,OHB SE
3,4.0,2022,Patrick Dekorsi,00:21:44.2,ACTEGA DS GmbH
4,5.0,5294,Lars Kuper,00:22:14.1,KPMG AG Wirtschaftsprüfungsgesellsc


In [35]:
women_run_df.head()

,Pos,Nr,Name,Zeit,Firma
0,1.0,2018,Katharina Saathoff,00:22:01.7,Absolute Run Bremen
1,2.0,2705,Camilla Zaage,00:23:48.1,AS Abrechnungsstelle AG
2,3.0,2632,Alina Reich,00:26:22.9,ArcelorMittal Bremen GmbH
3,4.0,189,Lea Brinkmann,00:27:06.4,ATLAS ELEKTRONIK GmbH
4,5.0,2759,Pia Nolte,00:27:17.0,Atermann König & Pavenstedt GmbH


In [36]:
def clean_int_col(df, colname):
    df[colname] = df[colname].astype(str).str.replace('.0', '').str.rstrip('.').astype(int)
    return df


def clean_time(df):
    df['Zeit'] = pd.to_timedelta(df['Zeit']).dt.total_seconds()
    return df

def clean_run_df(df):
    df = clean_int_col(df, 'Pos')
    df = clean_int_col(df, 'Nr')
    # leave in this order!
    # cleaning time before the other vars causes errors in other cols? black magic!
    df = clean_time(df)
    df = df.dropna()
    return df

In [37]:
men_run_df, women_run_df = clean_run_df(men_run_df), clean_run_df(women_run_df)
men_run_df['Geschlecht'] = 'M'
men_run_df['Geschlecht'] = men_run_df['Geschlecht'].astype('category')
women_run_df['Geschlecht'] = 'W'
women_run_df['Geschlecht'] = women_run_df['Geschlecht'].astype('category')

In [38]:
men_run_df.head()

,Pos,Nr,Name,Zeit,Firma,Geschlecht
0,1,4911,Michael Majewski,1156.7,hkk Krankenkassse,M
1,2,4022,Leonard Maisch,1212.7,DFKI GmbH - RIC,M
2,3,6074,Sven Zabel,1294.9,OHB SE,M
3,4,2022,Patrick Dekorsi,1304.2,ACTEGA DS GmbH,M
4,5,5294,Lars Kuper,1334.1,KPMG AG Wirtschaftsprüfungsgesellsc,M


In [39]:
women_run_df.head()

,Pos,Nr,Name,Zeit,Firma,Geschlecht
0,1,2018,Katharina Saathoff,1321.7,Absolute Run Bremen,W
1,2,2705,Camilla Zaage,1428.1,AS Abrechnungsstelle AG,W
2,3,2632,Alina Reich,1582.9,ArcelorMittal Bremen GmbH,W
3,4,189,Lea Brinkmann,1626.4,ATLAS ELEKTRONIK GmbH,W
4,5,2759,Pia Nolte,1637.0,Atermann König & Pavenstedt GmbH,W


In [40]:
combined_run_df = pd.concat([men_run_df, women_run_df], ignore_index=True)
combined_run_df['Geschlecht'] = combined_run_df['Geschlecht'].astype('category')

In [41]:
combined_run_df['VName'] = combined_run_df['Name'].str.split().str[:-1].apply(' '.join).str.strip()
combined_run_df['FName'] = combined_run_df['Name'].str.split().str[-1].str.strip()
combined_run_df['NoName'] = (combined_run_df['VName'] == 'No') & (combined_run_df['FName'] == 'Name')
combined_run_df['NoFName'] = (combined_run_df['FName'] == 'Noname') | (combined_run_df['NoName'])
combined_run_df['MTeam'] = combined_run_df['Firma'] == 'MERENTIS GmbH'
combined_run_df['MTeam'] = combined_run_df['MTeam'].astype(bool)
combined_run_df = combined_run_df.sort_values(by='Zeit').reset_index(drop=True)

In [42]:
combined_run_df.head(20)

,Pos,Nr,Name,Zeit,Firma,Geschlecht,VName,FName,NoName,NoFName,MTeam
0,1,4911,Michael Majewski,1156.7,hkk Krankenkassse,M,Michael,Majewski,False,False,False
1,2,4022,Leonard Maisch,1212.7,DFKI GmbH - RIC,M,Leonard,Maisch,False,False,False
2,11,7001,No Name,1212.8,Vitakraft pet care GmbH & Co. KG,M,No,Name,True,True,False
3,3,6074,Sven Zabel,1294.9,OHB SE,M,Sven,Zabel,False,False,False
4,4,2022,Patrick Dekorsi,1304.2,ACTEGA DS GmbH,M,Patrick,Dekorsi,False,False,False
5,1,2018,Katharina Saathoff,1321.7,Absolute Run Bremen,W,Katharina,Saathoff,False,False,False
6,5,5294,Lars Kuper,1334.1,KPMG AG Wirtschaftsprüfungsgesellsc,M,Lars,Kuper,False,False,False
7,7,2257,Daniel Schulze,1339.0,Airbus Operations GmbH,M,Daniel,Schulze,False,False,False
8,6,2439,Nikas Schröder,1339.0,ams.Solution AG,M,Nikas,Schröder,False,False,False
9,8,1521,Steffen Behrens,1339.5,Rheinmetall Electronics GmbH,M,Steffen,Behrens,False,False,False


In [43]:
men_df = combined_run_df[combined_run_df['Geschlecht'] == 'M']
men_df = men_df.sort_values(by='Pos', ascending=True).reset_index(drop=True)
jumps_men = men_df[men_df['Zeit'] < men_df['Zeit'].shift(1)].set_index('Pos')
jumps_men

,Nr,Name,Zeit,Firma,Geschlecht,VName,FName,NoName,NoFName,MTeam
Pos,,,,,,,,,,
11,7001,No Name,1212.8,Vitakraft pet care GmbH & Co. KG,M,No,Name,True,True,False
489,5192,Hubert Kuzma,1789.0,Jürgen Klose Industrietechnik GmbH,M,Hubert,Kuzma,False,False,False
748,3399,Alban Radloff,1864.0,cambio CarSharing,M,Alban,Radloff,False,False,False
1259,1710,Kristof Fiebing,1992.2,SV Werder Bremen,M,Kristof,Fiebing,False,False,False
1516,6684,No Name,2048.2,TenneT,M,No,Name,True,True,False
1771,1520,Martin Becker,2118.1,Rheinmetall Electronics GmbH,M,Martin,Becker,False,False,False
2540,921,Marcel Schindler,2382.0,HERMES Systeme GmbH,M,Marcel,Schindler,False,False,False
3308,1746,Uwe Lachermund,3015.2,swb runners,M,Uwe,Lachermund,False,False,False


In [44]:
prejump_times = men_df[men_df['Pos'].isin(jumps_men.index - 1)].sort_values('Pos')['Zeit'].reset_index(drop=True)
jump_times = jumps_men['Zeit'].reset_index(drop=True)
prejump_times.subtract(jump_times).to_frame().T

,0,1,2,3,4,5,6,7
Zeit,141.6,0.7,0.7,0.6,0.7,0.6,0.8,0.5


In [45]:
women_df = combined_run_df[combined_run_df['Geschlecht'] == 'W']
women_df = women_df.sort_values(by='Pos', ascending=True).reset_index(drop=True)
jumps_women = women_df[women_df['Zeit'] < women_df['Zeit'].shift(1)].set_index('Pos')
jumps_women

,Nr,Name,Zeit,Firma,Geschlecht,VName,FName,NoName,NoFName,MTeam
Pos,,,,,,,,,,
11,5740,Carina Hilker,1582.8,Mondelez,W,Carina,Hilker,False,False,False
123,4001,Maren König,1951.0,Deutsche Windtechnik AG,W,Maren,König,False,False,False
379,3350,Anna Boyke,2183.7,Team BUHLMANN RFS,W,Anna,Boyke,False,False,False
635,103,Natalja Gukenheimer,2340.2,Allianz Private Krankenversicherung,W,Natalja,Gukenheimer,False,False,False


In [46]:
prejump_times = women_df[women_df['Pos'].isin(jumps_women.index - 1)].sort_values('Pos')['Zeit'].reset_index(drop=True)
jump_times = jumps_women['Zeit'].reset_index(drop=True)
prejump_times.subtract(jump_times).to_frame().T

,0,1,2,3
Zeit,223.1,0.7,0.1,0.1


In [47]:
combined_run_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5651 entries, 0 to 5650
Data columns (total 11 columns):
 #   Column      Non-Null Count  Dtype   
---  ------      --------------  -----   
 0   Pos         5651 non-null   int64   
 1   Nr          5651 non-null   int64   
 2   Name        5651 non-null   object  
 3   Zeit        5651 non-null   float64 
 4   Firma       5651 non-null   object  
 5   Geschlecht  5651 non-null   category
 6   VName       5651 non-null   object  
 7   FName       5651 non-null   object  
 8   NoName      5651 non-null   bool    
 9   NoFName     5651 non-null   bool    
 10  MTeam       5651 non-null   bool    
dtypes: bool(3), category(1), float64(1), int64(2), object(4)
memory usage: 331.4+ KB


In [48]:
combined_run_df[combined_run_df.isna().any(axis=1)],
men_run_df[men_run_df.isna().any(axis=1)],
women_run_df[women_run_df.isna().any(axis=1)],


(Empty DataFrame
 Columns: [Pos, Nr, Name, Zeit, Firma, Geschlecht]
 Index: [],)

In [49]:
combined_run_df.to_parquet('/app/combined.parquet', index=False)